#### 개요
온라인으로 데이터를 받아, 데이터를 전처리 할 수 있도록 공사비 단위로 하나의 레코드로 취합하는 역활을 수행함
* STEP01: 학습데이터에서 특정 데이터를 추출해 온라인 입력형태의 데이터 샘플 생성`(이 작업은 실제 운영환경에서 실행되지 않음)`
* STEP02: JSON 데이터를 이용해 공사비별 전주/전선/인입선 갯 수 계산

In [1]:
import os
import sys
import json

import aidd.sys.config as cfg
from aidd.batch.data_manager import get_provide_data, read_data
from aidd.online.data_manager import get_online_sample

In [2]:
pdata = get_provide_data()

[84176d002a3b][2024-03-13 09:50:05.806465] 제공받은 데이터 불러오기 시작
[84176d002a3b][2024-03-13 09:50:34.543095]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:28.736443
[84176d002a3b][2024-03-13 09:50:59.384469]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:24.841164
[84176d002a3b][2024-03-13 09:51:30.308516]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:30.923903
[84176d002a3b][2024-03-13 09:51:43.154990]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:12.846323
[84176d002a3b][2024-03-13 09:51:43.155083] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:37.348640


#### STEP01: 온라인 시험 대상 추출

In [5]:
_dict = get_online_sample(data=pdata)

In [6]:
json_dict = {}
for key in _dict.keys():
    json_dict[key] = json.loads(_dict[key])

#### STEP02: 공사비별 전주/전선/인입선 갯 수 계산
* 전주/전선 갯 수가 10개 이상인 경우 해당 공사번호의 